# Quick Start Guide

A SoS script consists of one or more scripts, comments and optional SoS directives. In its simplest form, a sos script is simply a series of scripts that can be executed sequentially by different intepreters.

Let us assume that you are a bioinformaticist needed to compare the expression levels between two samples. After reading some online tutorials, you ended up with some working commands

```bash
# index reference genome
STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
    --genomeDir STAR_index
# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fasta --quantMode GeneCounts \
    --outFileNamePrefix aligned/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fasta --quantMode GeneCounts \
    --outFileNamePrefix aligned/mutated
```

The first command builds an index of the reference genome in preparation for the latter steps, the second command aligns reads from the first sample to the reference genome, and the third command aligns reads from the second sample to the reference genome. Do not panic if you do not know what these commands are doing, this is just an example.

These commands generate, among other files, two files named ``aligned/control.out.tab`` and ``aligned/mutated.out.tab`` with expression counts of all genes. You then wrote a [R](https://www.r-project.org/) script to analyze the results, something like

```R
control.count <- read.table('aligned/control.out.tab')
mutated.count <- read.table('aligned/mutated.out.tab')
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()
```

# Your first SoS script

The project completed successfully and you needed to archive the scripts for later reference. Instead of having two files lying around with perhaps another ``README`` file to describe what you have done, you can write a single SoS script named ``myanalysis.sos`` with content

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

run:
# index reference genome
STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
    --genomeDir STAR_index

# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fasta --quantMode GeneCounts \
    --outFileNamePrefix aligned/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fasta --quantMode GeneCounts \
    --outFileNamePrefix aligned/mutated

R:
control.count <- read.table('aligned/control.out.tab')
mutated.count <- read.table('aligned/mutated.out.tab')
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()
```

Here **`run`** and **`R`** are SoS actions that executes the following scripts in `bash` and `R` respectively. The scripts are included verbatim and end after reaching another SoS action or directive. It is however clearer to indent the script and write your SoS script as:

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

run:
    # index reference genome
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta\
        --genomeDir STAR_index

    # align reads to the reference genome
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn control.fasta --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn mutated.fasta --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()
```

The scripts in this case end with the end of indentation so that you can add some comments for the `R` script without being considered as part of the previous script.

You can execute the shell and R scripts defined in this SoS script sequentially by running command

```bash
sos run myanalysis
```

or simply

```bash
myanalysis.sos
```

if you give `myanalyis.sos` executable permission (`chmod +x myanalysis.sos`).

# Separate scripts into steps

Because the scripts perform different tasks, it is logically clearer to separate them into different **steps**. Actually, because the first reference-generating command is not a data processing step, it makes sense to separate it into two scripts. This can be done by inserting step headers to the script as follows:

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

[1]
# index reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
        --genomeDir STAR_index

[2]
# align reads to the reference genome
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn control.fasta --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn mutated.fasta --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[3]
# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()
```

Now, when you execute the script with command

```
sos run myscript
```

SoS will display `default_1`, `default_2` and `default_3` to report progress. The comments after section heads are considered step comments and will also be displayed during execution. 

# Make the script work for other input files

After a while, before you almost forgot about this analysis, you needed to analyze another pair of samples. You could copy ``myanalysis.sos`` to ``myanalysis2.sos``, change filenames and run it, but an easier way is to change your SoS file to accommodate other input files. This can be done by defining a command line argument and passing files name to a **SoS variable**:

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fasta formats. The first one for control sample
# and the second one for mutated sample.
parameter: fasta_files=['control.fasta', 'mutated.fasta']

[1]
# index reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
        --genomeDir STAR_index

[2]
# align reads to the reference genome
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${fasta_files[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${fasta_files[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[3]
# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()
```

A command line argument `fasta_files` is defined in with `parameter` keyword. With this definition, you can pass two filenames to variable `fasta_files` from command line

```bash
sos run myanalysis --fasta_files control1.fasta control2.fasta
```

`${fasta_files[0]}` and `${fasta_files[1]}` in command `STAR --genomeDir
...` will be replaced with their values before the commands are executed. Here `fasta_files[0]` and `fasta_files[1]` are Python expressions that will be evaluated during execution.

# Ignore steps that do not need to be rerun

Although the SoS script now accepts command line arguments, it is still no more than a compilation of scripts and you immediately realized that it is a waste of time to execute the first command each time. To solve this problem, you can convert the SoS script to a real workflow by telling SoS the input and output of each step:

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fasta formats. The first one for control sample
# and the second one for mutated sample.
parameter: fasta_files=['control.fasta', 'mutated.fasta']

[1]
# create a index for reference genome
output: 'STAR_index/chrName.txt'
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
        --genomeDir STAR_index

[2]
# align the reads to the reference genome
input:    fasta_files
depends:  'STAR_index/chrName.txt'
output:   ['aligned/control.out.tab', 'aligned/mutated.out.tab']
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[3]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()
```

Here we

- Use **output directive** to specify the expected output of all steps.
- Use **input directive** to specify the input of step 2. Step 1 by default has no input and input for step 3 by default is the output of step 2, its previous step.
- Use **depends directive** to let step 2 depend on the output of step 1.
- Use `${input[0]}` and `${input[1]}` in step 2 and 3 because these steps now have properly-defined `input`. This variable is defined by step input as the input file of the step.

With such information, when you run the same command

```bash
sos run myanalysis --input control1.fasta control2.fasta
```

SoS will ignore step 1 if this step has been run with output `STAR_index/chrName.txt`. The same happens to step 2 and 3 so all steps will be ignored if you run the script repeatedly with the same input and processing scripts. SoS uses **runtime signature** for each step and will re-run the step if and only if the content or filename of input, output files or the processing scripts are changed.

# Use make-rule to define resource-providing steps

Instead of using runtime signature to avoid re-running the first step, we can also make step 1 an optional step that will be executed only necessary. That is to say, we can define this step as an `auxiliary step` that will only be called when the file it **provides** does not exist.

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fasta formats. The first one for control sample
# and the second one for mutated sample.
parameter: fasta_files=['control.fasta', 'mutated.fasta']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
input:    fasta_files
depends:  'STAR_index/chrName.txt'
output:   ['aligned/control.out.tab', 'aligned/mutated.out.tab']
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()
```

The new script consists of two steps, and an auxiliary step `build_index` that will only be executed when `STAR_index/chrName.txt` is not available. A slight difference between this version and the previous one is that while the previous version will always execute the first step, this version will not execute it if `STAR_index/chrName.txt` has been generated before in any way, perhaps not by SoS.

# Execute long-running jobs externally

The first step will takes a long time to execute. Instead of executing them by SoS, you might want to submit the commands to a job-queue and be executed and monitored externally. This is especially useful when you need to execute multiple SoS workflows on a cluster-based system. Without going through the details on how to set up your job-queue (see another tutorial for details), it is almost trivial to modify your script to define part of a **step process** to a **task**:

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fasta formats. The first one for control sample
# and the second one for mutated sample.
parameter: fasta_files=['control.fasta', 'mutated.fasta']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
input:    fasta_files
depends:  'STAR_index/chrName.txt'
output:   ['aligned/control.out.tab', 'aligned/mutated.out.tab']
task:
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()
```

As you can see, the only difference is the insertion of `task:` directive before `run`. Now, when you execute the command, the `STAR` commands will be executed externally and you can monitor the status of the jobs using your web browser. If no job-queue is set up, the command will be executed in a separate process.

# Execute steps in parallel

Although the two steps of this example have to be executed sequentially, the first step runs the `STAR` command twice on two input files, and can be executed in parallel. You can tell this to SoS by modifying the script as follows

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fasta formats. The first one for control sample
# and the second one for mutated sample.
parameter: fasta_files=['control.fasta', 'mutated.fasta']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fasta \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
sample_type = ['control', 'mutated']
input:    fasta_files, group_by='single', paired_with='sample_type'
depends:  'STAR_index/chrName.txt'
output:   'aligned/${_sample_type}.out.tab'

task:     concurrent=True
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${_input!q} --quantMode GeneCounts \
        --outFileNamePrefix aligned/${_sample_type}

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()
```

Here we

1. Use option `group_by='single'` to pass input one by one to action. The action will be executed twice with `_input` set to the first and second input file respectively.
2. Define a variable `sample_type` and pair it with input files (option `paired_with`). This will generate a variable `_sample_type` for each input file so `_sample_type` will be `control` for the first input file, and `mutated` for the second.
3. Use `${_input}` and `${_sample_type}` to define partial `output`.
4. Use `${_input!q}` instead of `${_input}` in the script. This is a small trick to shell-quote filenames so that filenames with spaces and other special characters can be properly quoted in shell commands.

Now, if you execute the script with option `-j 2` (2 concurrent processes),

```bash
sos run myanalysis.sos --input control1.fasta control2.fasta -j 2
```

the second step submit two jobs to job-queue, or execute them in two separate processes if a job-queue is not setup.

We have showed you multiple versions of the same SoS script, each using more features of SoS. This actually demonstrates one of the advantages of the SoS system, namely you can start using SoS in minutes without knowing any of its advanced features, and gradually improve your script if needed.